# Parallel and Distributed Machine Learning

[Dask-ML](https://dask-ml.readthedocs.io) has resources for parallel and distributed machine learning.

## Types of Scaling

There are a couple of distinct scaling problems you might face.
The scaling strategy depends on which problem you're facing.

1. Large Models: Data fits in RAM, but training takes too long. Many hyperparameter combinations, a large ensemble of many models, etc.
2. Large Datasets: Data is larger than RAM, and sampling isn't an option.

![](static/ml-dimensions-color.png)

* For in-memory problems, just use scikit-learn (or your favorite ML library).
* For large models, use `dask_ml.joblib` and your favorite scikit-learn estimator
* For large datasets, use `dask_ml` estimators

## Scikit-Learn in 5 Minutes

Scikit-Learn has a nice, consistent API.

1. You instantiate an `Estimator` (e.g. `LinearRegression`, `RandomForestClassifier`, etc.). All of the models *hyperparameters* (user-specified parameters, not the ones learned by the estimator) are passed to the estimator when it's created.
2. You call `estimator.fit(X, y)` to train the estimator.
3. Use `estimator` to inspect attributes, make predictions, etc. 

Let's generate some random data.

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

In [ ]:
y[:8]

We'll fit a LogisitcRegression.

In [ ]:
from sklearn.svm import SVC

Create the estimator and fit it.

In [ ]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

Inspect the learned attributes.

In [ ]:
estimator.support_vectors_[:4]

Check the accuracy.

In [ ]:
estimator.score(X, y)

## Hyperparameters

Most models have *hyperparameters*. They affect the fit, but are specified up front instead of learned during training.

In [ ]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

In [ ]:
estimator.score(X, y)

## Hyperparameter Optimization

There are a few ways to learn the best *hyper*parameters while training. One is `GridSearchCV`.
As the name implies, this does a brute-force search over a grid of hyperparameter combinations.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

## Single-machine parallelism with scikit-learn

![](static/sklearn-parallel.png)

Scikit-Learn has nice *single-machine* parallelism, via Joblib.
Any scikit-learn estimator that can operate in parallel exposes an `n_jobs` keyword.
This controls the number of CPU cores that will be used.

In [ ]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

## Multi-machine parallelism with Dask

![](static/sklearn-parallel-dask.png)

Dask can talk to scikit-learn (via joblib) so that your *cluster* is used to train a model. 

In [ ]:
from pycon_utils import make_cluster
import dask_ml.joblib
from dask.distributed import Client

from sklearn.externals import joblib

In [ ]:
cluster = make_cluster()
cluster

In [ ]:
client = Client(cluster)
client

Let's try it on a larger problem (more hyperparameters).

In [ ]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    'kernel': ['rbf', 'poly', 'linear'],
    'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)

with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

In [ ]:
# clean up, when finished with the notebook
client.close()
cluster.close()